In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [6]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [ ]:
df_rumours.head()

In [7]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: Averaging history and use FFN

Here, we average the full history of a path and concatenate it to the current embedding (the total number of features that are passed into the FFN is 2 * sbert_embeddings.shape[0]).

Here, we will run the hyperparameter search to implement the FFN with the same parameters as the standard FFN baseline on the sentence embeddings.

In [11]:
num_epochs = 100
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [12]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=None,
    data_split_seed=123,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [01:43<00:00, 103.72s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


100%|██████████| 5568/5568 [00:09<00:00, 574.44it/s]


In [13]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id,input_dim
0,focal,0.758528,0.739237,"[0.8101623147494708, 0.6683107274969173]",0.366215,0.734007,0.722950,"[0.7782974742750235, 0.667601683029453]","[32, 32]",0.2,0.0010,0,2,False,0.0,768
0,focal,0.737881,0.718258,"[0.7926136363636364, 0.6439024390243903]",0.287907,0.740741,0.729192,"[0.7851162790697673, 0.6732673267326733]","[32, 32]",0.2,0.0010,1,2,False,0.0,768
0,focal,0.738779,0.718206,"[0.7943462897526502, 0.6420664206642068]",0.350734,0.742985,0.731275,"[0.787372330547818, 0.675177304964539]","[32, 32]",0.2,0.0010,12,2,False,0.0,768
0,focal,0.763016,0.740522,"[0.8169209431345353, 0.6641221374045801]",0.277131,0.736251,0.718457,"[0.789237668161435, 0.6476761619190404]","[32, 32]",0.2,0.0010,123,2,False,0.0,768
0,focal,0.744165,0.724017,"[0.7985865724381627, 0.6494464944649446]",0.462890,0.734007,0.722950,"[0.7782974742750235, 0.667601683029453]","[32, 32]",0.2,0.0010,1234,2,False,0.0,768
0,focal,0.727109,0.695288,"[0.7937584803256444, 0.596816976127321]",0.210524,0.716049,0.695778,"[0.7743086529884032, 0.6172465960665657]","[32, 32]",0.2,0.0001,0,2,False,0.1,768
0,focal,0.735188,0.708513,"[0.7966919365954515, 0.6203346203346204]",0.208805,0.714927,0.700275,"[0.7665441176470589, 0.6340057636887608]","[32, 32]",0.2,0.0001,1,2,False,0.1,768
0,focal,0.713645,0.682481,"[0.7819548872180452, 0.5830065359477125]",0.213860,0.703704,0.680732,"[0.7663716814159293, 0.5950920245398773]","[32, 32]",0.2,0.0001,12,2,False,0.1,768
0,focal,0.743267,0.716553,"[0.8035714285714286, 0.6295336787564767]",0.208321,0.722783,0.704435,"[0.7780772686433063, 0.6307922272047833]","[32, 32]",0.2,0.0001,123,2,False,0.1,768
0,focal,0.719030,0.692182,"[0.7830907830907831, 0.6012738853503184]",0.211335,0.705948,0.689172,"[0.761384335154827, 0.6169590643274853]","[32, 32]",0.2,0.0001,1234,2,False,0.1,768


In [15]:
ffn_mean_history.groupby(["learning_rate"]).mean()

/tmp/ipykernel_30409/4134656979.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_mean_history.groupby(["learning_rate"]).mean()


,accuracy,f1,valid_loss,valid_accuracy,valid_f1,dropout_rate,seed,gamma,k_fold,model_id,input_dim
learning_rate,,,,,,,,,,,
0.0001,0.727648,0.699003,0.210569,0.712682,0.694078,0.2,274.0,2.0,0.0,0.1,768.0
0.0005,0.749551,0.728822,0.290297,0.736476,0.724459,0.2,274.0,2.0,0.0,0.2,768.0
0.0010,0.748474,0.728048,0.348975,0.737598,0.724965,0.2,274.0,2.0,0.0,0.0,768.0


In [16]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,input_dim
0,focal,0.758528,0.739237,"[0.8101623147494708, 0.6683107274969173]",0.366215,0.734007,0.722950,"[0.7782974742750235, 0.667601683029453]","[32, 32]",0.2,0.001,0,2,False,768
0,focal,0.737881,0.718258,"[0.7926136363636364, 0.6439024390243903]",0.287907,0.740741,0.729192,"[0.7851162790697673, 0.6732673267326733]","[32, 32]",0.2,0.001,1,2,False,768
0,focal,0.738779,0.718206,"[0.7943462897526502, 0.6420664206642068]",0.350734,0.742985,0.731275,"[0.787372330547818, 0.675177304964539]","[32, 32]",0.2,0.001,12,2,False,768
0,focal,0.763016,0.740522,"[0.8169209431345353, 0.6641221374045801]",0.277131,0.736251,0.718457,"[0.789237668161435, 0.6476761619190404]","[32, 32]",0.2,0.001,123,2,False,768
0,focal,0.744165,0.724017,"[0.7985865724381627, 0.6494464944649446]",0.462890,0.734007,0.722950,"[0.7782974742750235, 0.667601683029453]","[32, 32]",0.2,0.001,1234,2,False,768


In [17]:
best_ffn_mean_history["f1"].mean()

0.7280477975493493

In [18]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.80252595, 0.65356964])